<center>
    <h1> ILI-286 - Laboratorio #2 </h1>
    <h2> Integración numérica aplicada </h2>
</center>


| Nombre | Rol | Email |
| :----- | :-- | :---- |
| Marco Rojas | 201073005-0 | marco.rojaso@alumnos.usm.cl |
| Hernán Vargas | 201073009-3 | hernan.vargas@alumnos.usm.cl |

## Tabla de contenidos
* [Introducción](#intro)
* [Desarrollo y analisís de resultados:](#desarrollo)
  1. [Integración Impropia](#se1)
  2. [The Caputo's Fractional Derivative](#se2)
* [Concluciones](#Concluciones)
* [Referencias](#Referencias)
* [Anexos](#Anexos) $\leftarrow$ Comenzar ejeciución aquí
  * [Implementación de algoritmos de integración](#an1)
  * [Implementación de la derivada fraccionaria](#an2)

<div id='intro' />
## Introducción


<div id='desarrollo'/>
## Desarrollo y analisís de resultados

<div id='se1'/>
## 1.- Integración Impropia

<div id='de1'/>
### 1.1) Cambio de variable propuesto
Si se quiere llevar la variable $t$ desde el intervalo de integración $[-\infty,x]$ a el $[a,b]$ se hace un cambio de 
variable utilizando la nueva variable $u$ definida por
$$ u = -\displaystyle \frac{b-a}{t-x-1} + a $$ 
$$ \Rightarrow t = \displaystyle \frac{b + a \, x + u (1+x)} {a-u} $$
$$ dt = \displaystyle \frac{b-a}{(u-a)^2} du$$
Graficamente:

In [ ]:
plt.figure(figsize=(12,6))
a = -1
b =  1
x =  3
t = np.linspace(-2*x, x, 200)
plt.plot(t, -(b-a)/(t-x-1)+a)
plt.title('Cambio de variable asumiendo a='+str(a) + " b=" + str(b) + " x=" + str(x))
plt.xlabel("variable t")
plt.ylabel("variable u")
plt.show()

Aplicando la transformación a la integral dada tenemos:
$$ f(x) = \int_{-\infty}^x \displaystyle \frac {1}{\sqrt{2 \pi}} \, e^{-  \frac{t^2}{2}} \, dt $$

$$ f(x) = \int_{a}^b \displaystyle \frac {1}{\sqrt{2 \pi}} \, e^{-  \frac{{(-b-a\,x + u(1+x)})^2}{2(u-a)^2}} \, \frac{b-a}{(u-a)^2} du $$
Como podemos ver, ahora la integral depende de $a$, $b$ y $x$. 
Definamos estas funciones para su posterior utilización:

In [ ]:
#Integrales:
int_orig = lambda t: (np.e**((-t**2)/2))/ np.sqrt(2*np.pi)
def get_int_abx(a, b, x):
    return lambda u: (np.e**(-(( (-b-a*x + u*(1+x)) / (u-a)  )**2 )/2)  * ((b-a)/(u-a)**2)  )/ np.sqrt(2*np.pi)

<div id='de2'/>
### 1.2) Complejidad algoritmica de los métodos  de integración
La implementación de los mismos puede analizarse en el [primer anexo](#an1).
Para cada una de las siguiente complejidades se asume una cantidad de intervalos $n$ y un costo de 
evaluación de la función de $o(f)$

##### 1.2.1) Midpoint Rule
 * Linspace: $1$ resta, $1$ división y $(n+1)$ multiplicaciones.
 * Diferencial: $1$ resta.
 * Puntos medios: $n$ multiplicaciones.
 * Evaluación: $n$ evaluaciones de la función.
 * Suma final: $n$ sumas y $1$ multiplicación.
 * **Total:** $1 + 1 + (n+1) + 1 + n + no(f) + n + 1 = 5 + 3n + no(f) = O(3n + no(f))$ 

##### 1.2.2) Trapezoid Method
 * Linspace + diferencial: $n + 4$ operaciones.
 * Calculo de la integral: $(n+1)$ evaluaciones de la función, $(n-1)$ multiplicaciones, $n+1$ 
 sumas y $2$ multiplicaciones.
 * **Total:** $n + 4 + (n+1)o(f) + (n-1) + (n+1) + 2 = 6 + 3n + (n+1)o(f) = O(3n + (n+1)o(f))$

##### 1.2.3) Simpson's Rule
 * Linspace + Diferencial:  $n+4$ operaciones.
 * Calculo de la integral: 
     * (n/2) evaluaciones en cada lado (derecha, centro y izquierda): 3(n/2) evaluaciones de $f$.
     * (n/2) multiplicaciones (del centro).
     * (n/2) multiplicaciones y una divición.
     * (n/2) sumas.
 * **Total:** $n+4 + \frac{3n}{2}o(f) +\frac{n}{2} + \frac{n}{2} +1 + \frac{n}{2} 
 = 5 + \frac{5n}{2} + \frac{3n}{2}o(f) = O(\frac{5n}{2} + \frac{3n}{2}o(f))$

##### 1.2.4) Gaussian Quadrature
 * Calculo de la integral: $n$ evaluaciones $n$ multiplicaciones y $n$ sumas: $no(f) + 2n$
 * La complejidad de la función que encuentra los puntos y sus pesos está determinada por el
 uso de `numpy.linalg.eigh` para una matriz de $n\times n$. Según la documentación[[1]](#ref1) 
 `eigh` utiliza las rutinas *LAPACK* `_syevd` y `_heevd` por lo que su complejidad debe ser 
 $O(n^3)$[[2]](#ref2).
 * **Total:** $2n + no(f) + n^3 = O(no(f) + n^3)$

<div id='de3'/>
### 1.3) Análisis de los métodos de integración

In [ ]:
#LEEME: Se supone que es una integral de una función de probabilidad así que no puede dar más de 1.
# quizás podamos hacer que si da más de 1 el metodo pare por que ya hay errores...
# Supongo que con esta pregunta lo que se busca es que respodamos por que da mal el número (no converge)
# y es debido a que los puntos están equiespaciados, por extrañas razones cuando incremento x
# al principio se pasa, pero luego se va a 0, supongo que pasa un error en algún momento. Este tambien
# podría ser un criterio de parada, así cuando se nota un comportamiento de-creciente se termina de ejcutar 
# dicho método... Puedes dejarme esto a mi.
N = 500
a = -1
b= 1
x = 100

#fun2 = lambda u: (np.e**(-(( (-b-a*x + u*(1+x)) / (u-a)  )**2 )/2)  * ((b-a)/(u-a)**2)  )/ np.sqrt(2*np.pi)
fun2 = get_int_abx(a,b,x)

print(midpoint(fun2, N, a,b))
print(gaussianquad(fun2, N, a, b))
#print(gaussianquad(fun2, N, -1, 1))

In [ ]:
## Acá hacer graficos mejor.
#matriz de 50 (alphas), 4 (métodos) y 5 h distintos
#res = np.zeros([50, 4, 5])
#delta = 10**-10
#for entre 0 y 49
#for i in range(0,50):
#    m =0;
#    for n in [10, 50, 100, 500, 1000]:
#        res[i, 0, m] = midpoint(fun2, n, a, b)
#        res[i, 1, m] = trapezoid(fun2, n, a+delta, b-delta)
#        res[i, 2, m] = simpsons(fun2, n, a+delta, b-delta)
#        res[i, 3, m] = gaussianquad(fun2, n, a, b)
#        m = m+1;
#
#print(res)

<div id='de4'/>
### 1.4) Caso $x = 0$
Sabemos que en $0$ deberíamos obtener $0.5$ pues estamos integrando la mitad de una función de probabilidad
simetrica.

In [ ]:
# TODO: aplicar truco del delta más abajo a las que fallan 
a = -1
b =  1
N =  500
func3 = get_int_abx(a, b, x=0)
for key in metodo:
    try:     print(key, "obtuvo:",metodo[key](func3, N, a,b))
    except:  print(key, "falló!")

Podemos observar como los metodos de Simpson y del trapecio fallaron devido a que en este caso
no podemos calcular los extremos. Aún así obtenemos el resultado correcto con el metodo del 
punto medio y la cuadratura Gaussiana, esta última sin error.

<div id='de5'/>
### 1.5) Caso $ x = +\infty$

Si se quiere llevar la variable $t$ desde el intervalo de integración $[-\infty,\infty]$ a el $[a,b]$ 
se hace un cambio de variable utilizando la nueva variable $u$ definida por:
$$ u = \displaystyle \frac{b-a}{\pi} \, \tan^{-1} (t) + \displaystyle \frac{b-a}{2} +a $$ 
$$ \Rightarrow  t = \displaystyle \cot ( \frac {\pi (u-b)}{a-b}) $$
$$ dt = - \displaystyle \frac {\pi \csc^2 ( \displaystyle \frac {\pi (u-b)}{a-b} )} {a-b} \, du $$

In [ ]:
plt.figure(figsize=(12,6))
a = -1
b =  1
x = np.inf;
t = np.linspace(-10, 10, 200)
plt.plot(t, ((b-a)*np.arctan(t))/np.pi + (b-a)/2 + a)
plt.title('Cambio de variable asumiendo a='+str(a) + " b=" + str(b) + " x=" + str(x))
plt.xlabel("variable t")
plt.ylabel("variable u")
plt.show()

Reemplazando en la integral nos queda:
$$ \lim_{x\rightarrow \infty}f(x) = \int_{-\infty}^\infty \displaystyle \frac {1}{\sqrt{2 \pi}} \, e^{-  \frac{t^2}{2}} \, dt $$

$$ = \int_a^b \displaystyle \frac {1}{\sqrt{2 \pi}} \, e^{- \displaystyle \frac{1}{2} \, \displaystyle \cot^2 ( \frac {\pi (u-b)}{a-b})}  \displaystyle \frac {-\pi \csc^2 ( \displaystyle \frac {\pi (u-b)}{a-b} )} {a-b} \, du  $$

Utilizando este cambio de variables para calcular la integral tenemos:

In [ ]:
N = 50
a = -1
b =  1
delta = 10**-15

func_inf_inf = lambda u: np.e**(-((  np.tan(np.pi * (u-b)/(a-b)  )**-2 )/2)) * (  (-np.pi *(np.sin(np.pi * (u-b)/(a-b)))**-2)/(a-b)  )/ np.sqrt(2*np.pi)

print(    midpoint(func_inf_inf, N, a,       b))
print(   trapezoid(func_inf_inf, N, a+delta, b-delta))
print(    simpsons(func_inf_inf, N, a+delta, b-delta))
print(gaussianquad(func_inf_inf, N, a,       b))

Podemos ver que todos los métodos dan resultados muy cercanos a $1$ que debería ser la 
respuesta de integrar una función de densidad de probabilidad en todo su dominio. 
El método de punto medio y la cuadratura Gaussiana no necesitaron ninguna modificación 
y tienen errores pequeños. Por otro lado en el método de Simpson y en el del trapecio se 
integró en un intervalo ligeramente más pequeño para evitar los errores que daría integrar
en los extremos.

<div id='se2'/>
## 2.- Derivación fraccionaria

<div id='de6'/>
### 2.1- Cambio de variable
Si se quiere llevar la variable $\tau$ desde el intervalo de integración $[a,t]$ a el $[-1,1]$ se hace un cambio de 
variable utilizando la nueva variable $u$ definida por
$$ u = \displaystyle \frac{a+t-2 \, \tau}{a-t}$$ 
$$ \Rightarrow \tau = \displaystyle \frac{a \, (1-u) + t \, (1+u)} {2} $$
$$ d \tau = \displaystyle \frac{t-a}{2} du$$
Graficamente:

In [ ]:
plt.figure(figsize=(12,6))
a = -2
t =  2
T = np.linspace(a, t, 200)
plt.plot(T, (a+t-2*T)/(a-t))
plt.title('Cambio de variable asumiendo a='+str(a) + " t=" + str(t))
plt.xlabel("variable Tau")
plt.ylabel("variable u")
plt.show()

Aplicando la transformación a la integral dada tenemos:
$$ \int_{a}^t \displaystyle \frac {f'(\tau)}{(t-\tau)^\alpha} \, d\tau $$

$$ = \int_{-1}^1 \displaystyle \frac {f'(\displaystyle \frac{a \, (1-u) + t \, (1+u)} {2})}{(t-\displaystyle \frac{a \, (1-u) + t \, (1+u)} {2})^\alpha} \, \displaystyle \frac{t-a}{2} du $$
Como podemos ver, ahora la integral depende de $a$, $t$ y $\alpha$. 
Definamos estas funciones para su posterior utilización:

In [ ]:
#Cálculo de la integral de la derivada fraccionaria, devuelve una función que depende te de t.
t= sp.Symbol('t', real=True, positive=True, imaginary=False)
u= sp.Symbol('u', real=True, positive=True, imaginary=False)
N = 50
def get_int_df(fun, a, alpha):
    g = sp.diff(fun, t);
    #g_eval = (g.subs(t, t-((a*(1-u)+t*(1+u))/sp.Integer(2))) / ((a*(1-u)+t*(1+u))/sp.Integer(2) -a )**alpha) * (t-a)/sp.Integer(2)
    g_eval = (g.subs(t, ((a*(1-u)+t*(1+u))/sp.Integer(2))) / (t- (a*(1-u)+t*(1+u))/sp.Integer(2)  )**alpha) * (t-a)/sp.Integer(2)
    
    f_int = gaussianquad_sympy(g_eval, N, -1, 1)
    
    #tomamos el resultado real, con la raíz positiva
    f_int = f_int.subs(sp.zoo, 2)
    return f_int


<div id='de7'/>
### 2.2- Implementación de fraction_diff

In [ ]:
t= sp.Symbol('t', positive=True, real=True, imaginary=False)
f = t**4 
def fraction_diff(f, t0, alpha, a):
    tau = sp.Symbol('Tau', positive = True, real=True, imaginary=False)
    #calculamos el valor de n como el piso de alpha
    n = sp.Integer(sp.floor(alpha))
    #calculamos el valor del nuevo alpha restandole n
    frac = sp.Rational(alpha - n,1)
    
    fr_diff = f
    #si n es dinstinto de cero entonces derivamos la función n veces
    if (n != 0):
        fr_diff = sp.diff(f, t, n)
    
    #una vez derivada n veces la función procedemos a realizar la derivada fraccionaria
    F_tau = get_int_df(fr_diff, a, frac)
    
    #dividimos la derivada fraccionaria por la función gamma
    fr_diff = F_tau/sp.gamma(1-frac)
    
    #evaluamos en t0
    return fr_diff.subs(t, t0)
  
print(sp.simplify(fraction_diff(f, 0.9, 1.5, 0)).evalf())

.<div id='de8'/>
### 2.3- Cálculos

In [ ]:
t= sp.Symbol('t', positive=True, real=True, imaginary=False)
# Estas funciones se quedna pegadas algunas
f = 20;
print(fraction_diff(f, sp.Rational(3,10), sp.Rational(2,10), 0).evalf())

f = sp.sin(t);
print(fraction_diff(f, sp.Rational(2,1), sp.Rational(15,10), 0).evalf())

f = sp.exp(t);
print(fraction_diff(f, sp.Rational(60,1), sp.Rational(464,10), 0).evalf())

f = t;
print(fraction_diff(f, sp.Rational(8,10), sp.Rational(7,10), 0).evalf())

f = sp.log(t);
print(fraction_diff(f, sp.Rational(4,10), sp.Rational(3,10), 0).evalf())

f = t**2;
print(fraction_diff(f, sp.Rational(3, 1), sp.Rational(5,10), 0).evalf())

f = sp.tanh(t);
print(fraction_diff(f, sp.Rational(3,1), sp.Rational(5,10), 0).evalf())

## Conclusiones

## Referencias
<div id='ref1'\> [1] [NumPy v1.10 Manual](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.linalg.eigh.html) numpy.linalg.eigh. *The eigenvalues/eigenvectors are computed using LAPACK routines _syevd, _heevd*. Revisado 22/10/2015

<div id='ref2'\> [2] [LAPACK Benchmark](http://www.netlib.org/lapack/lug/node71.html). Ver tabla 3.13. Revisado el 22/10/2015

## Anexos
Comenzamos cargando los modulos necesarios:

In [ ]:
import matplotlib
%matplotlib inline
import numpy as np
import sympy as sp
import scipy as sy
import matplotlib.pyplot as plt

<div id='an1'\> 
### Implementación de algoritmos de integración:
Las siguientes funciones reciben como parámetros de entrada una función $f(x)$ en la variable `func`, un número de intervalos `N` y los números `a` y `b` que serán el rango de integración.

Retornan la estimación numerica de $\int_a^b f(x)\,dx$ mediante el método seleccionado.
#### Midpoint Rule

In [ ]:
def midpoint(fun, N, a, b):
    #Vectorizamos la funcion para poder llamarla con muchos puntos a la vez.
    f_vec = np.vectorize(fun)
    #Obtenemos N intervalos y N+1 puntos equidistantes entre a y b.
    x = np.linspace(a, b, N+1)
    #Obtenemos el diferencial de distancia entre los puntos.
    dx = x[1]-x[0]
    #Obtenemos los puntos medios del intervalo.
    midpoints = x[:-1] + 0.5*dx
    #Evaluamos en los puntos
    midpoint_values = f_vec(midpoints)
    #Calculamos y retornamos el resultado final
    return sum(midpoint_values*dx)

#### Trapezoid Method

In [ ]:
def trapezoid(fun, N, a, b):
    f_vec = np.vectorize(fun)
    x     = np.linspace(a, b, N+1)
    dx    = x[1]-x[0]
    xmiddle = x[1:-1] #Los puntos que no estan a los extremos del intervalo.
    return 0.5*dx*(f_vec(x[0])+2*sum(f_vec(xmiddle))+f_vec(x[-1]))

#### Simpson's Rule

In [ ]:
def simpsons(fun, N, a, b):
    f_vec = np.vectorize(fun)
    x     = np.linspace(a, b, N+1)
    if N%2==1:
        print("Simpsons solo aplica con una cantidad de intervalos pares.")
        return
    dx = x[1]-x[0]
    #Obtenemos elementos de la izquierda partiendo desde el primero de dos en dos hasta el antepenultimo.
    xleft   = x[:-2:2]
    #Obtenemos elementos centrales partiendo desde el segundo de dos en dos.
    xmiddle = x[1::2]
    #Obtenemos elementos de la derecha partiendo desde el tercero de dos en dos.
    xright  = x[2::2]
    #Los sumamos y multiplicamos. La multiplicación por 2 es implicita pues xleft y xright contienen los 
    # mismos elementos menos el primero y el último.
    return sum((f_vec(xleft)+4*f_vec(xmiddle)+f_vec(xright))*(dx/3))

#### Gaussian Quadrature

In [ ]:
def gaussianquad(fun, N, a, b):
    print(fun)
    f_vec = np.vectorize(fun)
    print(f_vec)
    #Calculamos los pesos y los nodos por el metodo visto en el ipython 'U2_QuadWorld.ipynb'
    x, w = gaussian_nodes_and_weights(N, a, b)
    print(f_vec(x))
    return sum( w * f_vec(x) )

# Función gaussiana idéntica a gaussianquad solo que utiliza sintaxis de sympy para utilizarse con funciones de 
# sympy en vez de funciones lambda
def gaussianquad_sympy(fun, N, a, b):
    #Calculamos los pesos y los nodos por el metodo visto en el ipython 'U2_QuadWorld.ipynb'
    x, w = gaussian_nodes_and_weights(N, a, b)
    val = 0;
    for i in range(0, x.size):
        val = val + w[i]*fun.subs(u, x[i])
    return val

def gaussian_nodes_and_weights(N, a, b):
    if N==1: 
        return np.array([1]), np.array([2])
    beta = .5 / np.sqrt(1.-(2.*np.arange(1.,N))**(-2))
    T = np.diag(beta,1) + np.diag(beta,-1)
    D, V = np.linalg.eigh(T)
    x = D
    #Reescalamiento -1, 1
    x = .5 * ( (b-a)*x + b + a)
    w = 2*V[0,:]**2
    w = .5*(b-a)*w
    return x, w

<div id='an2'\> 
### Implementación de la derivada fraccionaria

### Utilidades

In [ ]:
metodo = {
    "Punto Medio": midpoint,
    "Trapecio": trapezoid,
    "Simpson": simpsons,
    "Cuadratura Gaussiana": gaussianquad
}